# Geomorphometry in Julia 

We discuss (global) elevation modelling, including recent advances, and follow up with a hands-on session in Julia, where we will use Rasters.jl and Geomorphometry.jl to download, analyze and visualize a subset of a global elevation model. These operations include hydrological functions such as flow accumulation methods, and the differences between several common algorithms. The importance of the correct coordinate reference systems and cellsize of a raster will be demonstrated.

Key (geospatial) concepts in Julia will be introduced, including the performance gains made possible with Julia, although prior exposure (can be by watching previous lectures) is expected. An introduction lecture to Julia will be given beforehand.


# Open and crop a DTM
Here we'll be using the GEDTM30 dataset, and crop it to my favourite holiday area in Austria.

In [ ]:
using Rasters
using ArchGDAL

fn = "https://s3.opengeohub.org/global/edtm/gedtm_rf_m_30m_s_20060101_20151231_go_epsg.4326.3855_v1.2.tif"
gedtm30 = Raster(fn, lazy=true)  # lazy is essential here!

In [ ]:
metadata(gedtm30)

In [ ]:
using Extents

extent = Extent(X=(9.8, 10.0), Y=(47.3,47.5))
voralberg = crop(gedtm30, to=extent) ./ 10 # !!! not in metadata, but data is scaled


Let's read the data into memory, so we can operate faster on it.

In [ ]:
voralberg = read(voralberg)
Rasters.write("voralberg.tif", voralberg, force=true)

# Plot
Let's inspect our DTM to see what we're dealing with.

In [ ]:
using CairoMakie

plot(voralberg, colormap=:turbo)  # a colormap that has a large range

# Geomorphometry
To analyse the elevation model, let's use the Geomorphometry.jl package, which provides a range of tools for terrain analysis. You can find its complete documentation here: https://deltares.github.io/Geomorphometry.jl/dev/.

Let's start with making a hillshade visualisation, as this can help us understand the terrain better.

In [ ]:
using Geomorphometry

plot(hillshade(voralberg))

Ah, we hit an error. For now, the package doesn't handle missing data well, so let's fil the missing values.

In [ ]:
voralberg = coalesce.(voralberg, minimum(skipmissing(voralberg)))

In [ ]:
plot(hillshade(voralberg))

A hillshade is not really realistic, so let's illuminate the terrain with specific (artificial) sun angles.

In [ ]:
plot(multihillshade(voralberg, azimuth = [90, 180, 270], zenith=30))

Another method to visualise terrain is the progressive slope shading method (PSSM), used in archeology to emphasize relative flat areas.

In [ ]:
f = plot(pssm(voralberg), colormap=:Greys)
# save("voralberg_pssm.png", f)  # this is how you would save a the picture
f

# Slope
Given the visualisation above already uses the slope, let us calculate it directly. We see it's really similar to the PSSM result, but PSSM darkens slopes much more.

In [ ]:
plot(slope(voralberg), colormap=:Greys)

## Automatic cellsize
To calculate the slope, one uses rise over run, but you might've noticed that this is a geographic raster (latitude and longitude), but the elevation is in meters. Geomorphometry automatically calculates the correct cellsize for you. We can inspect that with `cellsize`:

In [ ]:
Geomorphometry.cellsize(voralberg)

And we can see the impact of not using the correct cellsize, with setting the `cellsize` kwarg to `(1,1)`, the degrees are interpreted as meters, resulting in very steep slopes.

In [ ]:
plot(slope(voralberg, cellsize=(1,1)))

The cellsize is used for all cells, which is better than nothing, but note that this is still a simplification. In reality, geographic rasters can have varying cell sizes, especially when covering larger areas. We can see this by visualizing the cellsize across the raster, where the raster sizes becomes smaller towards the poles (higher latitudes).


In [ ]:
import Proj
plot(cellarea(voralberg))

## Multiple algorithms
Geomorphometry supports multiple algorithms for analyses like slope, aspect, and others, enabling a deeper understanding of the terrain and its features. For example, there are three options for calculating slope:

In [ ]:
A = slope(voralberg, method=ZevenbergenThorne())
B = slope(voralberg, method=Horn())
C = slope(voralberg, method=Geomorphometry.MaximumDownwardGradient())

f = Figure(size=(900, 300))
a = Axis(f[1, 1], aspect=AxisAspect(1), title = "ZevenbergenThorne")
b = Axis(f[1, 2], aspect=AxisAspect(1), title = "Horn")
c = Axis(f[1, 3], aspect=AxisAspect(1), title = "MDG")

plot!(a, A)
plot!(b, B)
plot!(c, C)
f

# Aspect
The same is true for aspect, which is the direction the slope faces (so full 360 degrees).

In [ ]:
plot(aspect(voralberg; method=Horn()); colormap=:romaO)

# Curvature
Curvature describes how much the slope changes, and can be useful to identify ridges and valleys.

In [ ]:
# heatmap(profile_curvature(voralberg); colorrange=(-0.1,0.1), colormap=:tarn)
# heatmap(plan_curvature(voralberg); colorrange=(-0.1,0.1), colormap=:tarn)
# heatmap(tangential_curvature(voralberg); colorrange=(-0.1,0.1), colormap=:tarn)
heatmap(laplacian(voralberg); colorrange=(-0.1,0.1), colormap=:tarn)

# Relative position
There are several terrain descriptors that can be used to analyze the relative position of a point with respect to its neighbors. These include TPI, TRI, RIE, BPI, rugosity and roughness.

In [ ]:
plot(TPI(voralberg), colormap=:delta, colorrange=(-10,10))


This output is a bit noisy, as there's noise in the GEDTM30 (any global dataset). For better results we can use a larger custom sized Window operation (from [Stencils.jl](https://rafaqz.github.io/Stencils.jl/dev)) to reduce the impact of this noise.

In [ ]:
Geomorphometry.Moore(1)  # this is the default 8 neighbors

In [ ]:
window = Geomorphometry.Annulus(4, 3)  # no direct neighbors at all!

Using this larger window produces much clearer results:

In [ ]:
plot(TPI(voralberg, window); colormap=:delta, colorrange=(-10,10))

# Hydrology
Geomorphometry has some methods for hydrology, including flow accumulation. There are again several algorithms available:

In [ ]:
v = voralberg[1:200, 1:200]  # we zoom in a bit

accD8, lddD8 = flowaccumulation(v; method=D8())
accDInf, lddDInf = flowaccumulation(v; method=DInf())
accFD8, lddFD8 = flowaccumulation(v; method=FD8())

f = Figure(size=(1200, 400))
a = Axis(f[1, 1], aspect=AxisAspect(1), title = "D8")
b = Axis(f[1, 2], aspect=AxisAspect(1), title = "DInf")
c = Axis(f[1, 3], aspect=AxisAspect(1), title = "FD8")

plot!(a, log10.(accD8), colormap=:rain, colorrange=(3,10))
plot!(b, log10.(accDInf), colormap=:rain, colorrange=(3,10))
plot!(c, log10.(accFD8), colormap=:rain, colorrange=(3,10))
f


# Going beyond
There's more to explore in Geomorphometry.jl, for example, filters for classification, directional slope and curvature, and cost-distance analysis, but this has to do for now. Please check out the documentation at https://deltares.github.io/Geomorphometry.jl/.

*Finally, don't forget that open-source only works if you take part of it! Please report bugs, issues, and feature requests at Github.*